In [ ]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List

class NewsAnalysisResult(BaseModel):
    is_follow_up: bool
    reason: str

client = genai.Client(api_key="xxx")

def analyze_news_relationship(original_news: str, original_news_tags: List[str], 
                             follow_up_news: str):
    tags_str = ", ".join(original_news_tags)
    
    contents = f"""
        You are a news analyst evaluating news continuity with strict criteria.

        Task: Methodically analyze if BOTH conditions are met, following these exact steps:

        CONDITION 1 - Subsequent Report Check:
        - Does the follow-up news reference the same core event as the original?
        - Does it contain newer information or developments about this event?
        - Does it explicitly or implicitly acknowledge the original news?
        If ALL above are true, Condition 1 is met.

        CONDITION 2 - Tag Matching Check:
        - Extract the central topic/theme of both news items
        - Does the follow-up news clearly belong to at least one of the categories 
          defined by the original tags?
        - Is the central focus of the follow-up news related to at least one of these tags?
        If ALL above are true, Condition 2 is met.

        You must return EXACTLY one of these responses:
        - RESULT: TRUE (only if both conditions are fully satisfied)
        - RESULT: FALSE (if either condition fails)

        Original news: {original_news}
        Original news tags: {tags_str}
        Follow-up news: {follow_up_news}
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[contents],
        config={
            'response_mime_type': 'application/json',
            'response_schema': NewsAnalysisResult,
            'temperature': 0,
        }
    )
    
    return response.parsed

In [ ]:
original_news = """....The European Union, with its 15 member countries and 373 million citizens, is not the """
original_news_tag = ["ec general", "european community"]
follow_up_news = """....United Nations says worlds population is steadily getting older everywhere; Joseph Chamie, demographer who directs UN population division..."""

result = analyze_news_relationship(original_news, original_news_tag, follow_up_news)

In [ ]:
result.is_follow_up